<a href="https://colab.research.google.com/github/ykhier/Cloud_Course/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
import pandas as pd
from ipywidgets import widgets, VBox, Output
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option("display.max_columns", None)
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)
data_df = pd.DataFrame(data['result']['records'])
data_df.head()
output_area = Output()


In [ ]:
# Create widgets
tozeret_nm_dropdown = widgets.Dropdown(
    options=[''] + sorted(data_df['tozeret_nm'].unique().tolist()),
    description='Tozeret:',
    style={'description_width': 'initial'}
)

kinuy_mishari_dropdown = widgets.Dropdown(
    options=[''],
    description='Kinuy Mishari:',
    style={'description_width': 'initial'}
)
# Function to update the second dropdown based on the selection of the first
def update_kinuy_mishari_options(change):
    if change['new']:  # Check if a valid option is selected
        filtered_values = data_df[data_df['tozeret_nm'] == change['new']]['kinuy_mishari'].unique()
        kinuy_mishari_dropdown.options = [''] + sorted(filtered_values)
    else:
        kinuy_mishari_dropdown.options = ['']

In [ ]:
def update_output(change=None):
    output_area.clear_output()
    selected_tozeret = tozeret_nm_dropdown.value
    selected_kinuy = kinuy_mishari_dropdown.value

    if selected_tozeret and selected_kinuy:
        filtered_df = data_df[
            (data_df['tozeret_nm'] == selected_tozeret) &
            (data_df['kinuy_mishari'] == selected_kinuy)
        ]
        total_records = len(filtered_df)
        unique_ramat_gimur = filtered_df['ramat_gimur'].astype(str).unique()

        with output_area:
            print(f"Total Records: {total_records}")
            print(
                "Unique Ramat Gimur: "
                f"{', '.join(unique_ramat_gimur) if len(unique_ramat_gimur) > 0 else 'None'}")
    else:
        with output_area:
            print("Please select valid options for both dropdowns.")


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from plotly.offline import iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns  #seaborn for heatmap

df = pd.DataFrame(data)


In [ ]:
import matplotlib.pyplot as plt
def graph():
  year_counts = (
     data_df
    .groupby('shnat_yitzur')['mispar_rechev']
    .nunique()
  )

  plt.figure(figsize=(10, 4))
  plt.bar(year_counts.index.astype(str), year_counts.values)
  plt.xlabel('Year')
  plt.ylabel('Number of different cars')
  plt.title('Number of different cars produced each year')
  plt.xticks(rotation=90)
  plt.tight_layout()
  plt.show()


In [ ]:
def staticticsGraph():
    plt.figure(figsize=(10, 4))
    sns.histplot(data_df['shnat_yitzur'], bins=20, kde=True)
    plt.title('Date Distribution')
    plt.xlabel('Age')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
tozeret_nm_dropdown.observe(update_kinuy_mishari_options, names='value')
kinuy_mishari_dropdown.observe(update_output, names='value')
display(VBox([tozeret_nm_dropdown, kinuy_mishari_dropdown, output_area]))

In [ ]:
# Tab 1: Data Overview
tab1_content = widgets.Output()
with tab1_content:
    print("statistics:")
    display(data_df.describe())
    staticticsGraph()
# Tab 2: Raw Data
tab2_content = widgets.Output()
with tab2_content:
    print("Data:")
    display(data_df)

# Tab 3: Graph
tab3_content = widgets.Output()
with tab3_content:
    print("graph:")
    graph()

tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content])
tabs.set_title(0, 'Tab 1: Statistics')
tabs.set_title(1, 'Tab 2: Data')
tabs.set_title(2, 'Tab 3: Graph')

display(tabs)
